In [2]:
%matplotlib notebook
import numpy as np
from numpy.fft import fft2, ifft2, fftshift
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import scipy
from scipy.ndimage import gaussian_filter1d as gf1d
from scipy.ndimage import gaussian_filter as gf
from scipy.ndimage import uniform_filter as uf

import sys
sys.path.append("../kai_colloids/PyDDM") #must point to the PyDDM folder
import ddm_analysis_and_fitting as ddm

import tiff_file
import ddm_clean 

import io 
import sys
import csv
import os
import glob #glob is helpful for searching for filenames or directories
import pickle #for saving data

In [3]:
def last_3chars(x):
    return((x.split('\\')[-1])[-7:-4])

In [19]:
###################################################################################
# Specify where the data is and file name for image (or image sequence)           #
###################################################################################

directory = "Z"
exp = "10-15-22_s2_theBigOne2"
data_dir = directory+":\\Gregor L\\__Kai Colloids\\"+exp+"\\all tiff files\\" 
data_dir2 = directory+":\\Gregor L\\__Kai Colloids\\"+exp+"\\reordered tiffs\\" #bottom_row_t01\\
plot_saveto= directory+":\\Gregor L\\__Kai Colloids\\SIA\\Data\\"+exp+"\\"

files = glob.glob(data_dir+"*_t*")
#files = sorted(files, key = last_3chars)
print("found %i files" % len(files))
print(int((len(files))/3))
for i,f in enumerate(files): print (' %i \t %s' % (i, f.split('\\')[-1]))

found 54 files
18
 0 	 row1_t1.tif
 1 	 row1_t2.tif
 2 	 row1_t3.tif
 3 	 row1_t4.tif
 4 	 row1_t5.tif
 5 	 row1_t6.tif
 6 	 row1_t7.tif
 7 	 row1_t8.tif
 8 	 row1_t9.tif
 9 	 row2_t1.tif
 10 	 row2_t2.tif
 11 	 row2_t3.tif
 12 	 row2_t4.tif
 13 	 row2_t5.tif
 14 	 row2_t6.tif
 15 	 row2_t7.tif
 16 	 row2_t8.tif
 17 	 row2_t9.tif
 18 	 row3_t1.tif
 19 	 row3_t2.tif
 20 	 row3_t3.tif
 21 	 row3_t4.tif
 22 	 row3_t5.tif
 23 	 row3_t6.tif
 24 	 row3_t7.tif
 25 	 row3_t8.tif
 26 	 row3_t9.tif
 27 	 row4_t1.tif
 28 	 row4_t2.tif
 29 	 row4_t3.tif
 30 	 row4_t4.tif
 31 	 row4_t5.tif
 32 	 row4_t6.tif
 33 	 row4_t7.tif
 34 	 row4_t8.tif
 35 	 row4_t9.tif
 36 	 row5_t1.tif
 37 	 row5_t2.tif
 38 	 row5_t3.tif
 39 	 row5_t4.tif
 40 	 row5_t5.tif
 41 	 row5_t6.tif
 42 	 row5_t7.tif
 43 	 row5_t8.tif
 44 	 row5_t9.tif
 45 	 row6_t1.tif
 46 	 row6_t2.tif
 47 	 row6_t3.tif
 48 	 row6_t4.tif
 49 	 row6_t5.tif
 50 	 row6_t6.tif
 51 	 row6_t7.tif
 52 	 row6_t8.tif
 53 	 row6_t9.tif


In [20]:
###### # specify each condition (frame_names) and the time points of data collection (time_array) #
############################################################################################
frame_names = ["1-3 kA-WT", "WT (no kA)", "EA KaiC", "AE KaiC"]
time_array =  [1.1, 4.0, 7.4, 10.3, 14.7, 17.7, 20.3, 23.8, 27.0]
#[0.5, 4.5, 8, 12, 20, 24]
#[0.5, 3.5, 6.5, 9.3, 12.1, 15, 18, 21, 24.5, 27.3, 40.5, 44.3]  
#[0.5, 3.2, 17.75, 20, 22, 24.25, 26, 41.75, 44]
#[0.85, 3.5, 18, 20.4, 22.25, 24.5, 26.2, 41.75, 44.2]
#s1 [0.67, 3.67, 6.67, 10, 12.83, 17.67, 21.67, 24.92, 28.17, 45.67, 49.08, 69.25]
#s2 [1, 3.83, 6.83, 10.17, 13.08, 18, 21.92, 25.25, 28.67, 45.92, 49.33, 69.42]
#s1 [1.1, 4.0, 7.4, 10.3, 14.7, 17.7, 20.3, 23.8, 27.0]
#s2 [1.2, 4.4, 7.8, 10.8, 15.2, 18.3, 20.8, 24.3, 28.0]
fig_size = 10,10/1.618
font_size = 16
dpi_num = 800
pixel_size = 0.364 # 4*0.091 = 0.364

print((files[0]))
eg_im= tiff_file.imread(files[0])
print(eg_im.shape)
frames_per_tiff = int(eg_im.shape[0])
print(frames_per_tiff)
arr_length = int((len(files))/3)
print(arr_length)

Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t1.tif
(5, 1440, 1920)
5
18


In [21]:
###################################################################################################
# makes new tiff files, each new tiff has all consecutive time points for each specific condition #
###################################################################################################
arr_length = int((len(files))/6)

for j in range(frames_per_tiff-1):
    frame_num = str(j+1)

    for i in range(arr_length):    
        row1_im = tiff_file.imread(files[i],key=[j])
        row2_im = tiff_file.imread(files[i+arr_length],key=[j])
        row3_im = tiff_file.imread(files[i+(arr_length*2)],key=[j])
        row4_im = tiff_file.imread(files[i+(arr_length*3)],key=[j])
        row5_im = tiff_file.imread(files[i+(arr_length*4)],key=[j])
        row6_im = tiff_file.imread(files[i+(arr_length*5)],key=[j])
        #print(br_frame1_im.shape[0])
        print(files[i])
        print(files[i+(arr_length*5)])
        if i==0:
            row1 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row2 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row3 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row4 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row5 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row6 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
        row1[i] = row1_im
        row2[i] = row2_im
        row3[i] = row3_im
        row4[i] = row4_im
        row5[i] = row5_im
        row6[i] = row6_im

    tiff_file.imsave(data_dir2 +'row1_'+frame_names[j]+'.tif', row1)
    tiff_file.imsave(data_dir2 +'row2_'+frame_names[j]+'.tif', row2)
    tiff_file.imsave(data_dir2 +'row3_'+frame_names[j]+'.tif', row3)
    tiff_file.imsave(data_dir2 +'row4_'+frame_names[j]+'.tif', row4)
    tiff_file.imsave(data_dir2 +'row5_'+frame_names[j]+'.tif', row5)
    tiff_file.imsave(data_dir2 +'row6_'+frame_names[j]+'.tif', row6)
    print(row1.shape)

Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t1.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row6_t1.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t2.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row6_t2.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t3.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row6_t3.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t4.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row6_t4.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t5.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row6_t5.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t6.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row6_t6.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s2_theBigOne2\all tiff files\row1_t7.tif

In [8]:
###################################################################################################
# makes new tiff files, each new tiff has all consecutive time points for each specific condition #
###################################################################################################
arr_length = int((len(files))/3)

for j in range(frames_per_tiff-1):
    frame_num = str(j+1)

    for i in range(arr_length):    
        row1_im = tiff_file.imread(files[i],key=[j])
        row2_im = tiff_file.imread(files[i+arr_length],key=[j])
        row3_im = tiff_file.imread(files[i+(arr_length*2)],key=[j])
        #print(br_frame1_im.shape[0])
        print(files[i])
        if i==0:
            row1 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row2 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row3 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
        row1[i] = row1_im
        row2[i] = row2_im
        row3[i] = row3_im

    tiff_file.imsave(data_dir2 +'row1_'+frame_names[j]+'.tif', row1)
    tiff_file.imsave(data_dir2 +'row2_'+frame_names[j]+'.tif', row2)
    tiff_file.imsave(data_dir2 +'row3_'+frame_names[j]+'.tif', row3)
    print(row1.shape)

Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t1.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t2.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t3.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t4.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t5.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t6.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t7.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t8.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row1_t9.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row2_t1.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row2_t2.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row2_t3.tif
Z:\Gregor L\__Kai Colloids\10-15-22_s1_theBigOne2\all tiff files\row2_t4.tif

In [28]:
#####################################################
# **optional** preview of tiff files to be analyzed #
#####################################################
tiff_name = 'row3_'+frame_names[0]
tiff_to_show = tiff_file.imread(data_dir2 +tiff_name+'.tif')

fig = plt.figure(figsize=(10,10))    
for i in range (arr_length):
    ax = plt.subplot(4,4,i+1) ##change according to number of frames (6 --> 3,3; 12 --> 4,4)
    ax.set_title("~" + str(time_array[i]) + " hrs", fontsize=10)
    ax.matshow(tiff_to_show[i], cmap = 'gray') #fig.set_cmap('hot') 
    plt.axis('off')
    plt.tight_layout(pad=.25)
fig.savefig(data_dir2+tiff_name+".jpg", dpi=(dpi_num))

TypeError: len() takes exactly one argument (0 given)